# 一 线程理论
进程指的是一个程序正在进行，线程可以理解为是一条流水线的工作过程，我们会把一个进程比如成一个车间，而线程就比喻成车间里面的流水线，很明显一个车间里面至少应该有一条流水线，确实，一个进程内就自带一个线程。在这里先把我们之前学过的关于进程的概念忘记，因为准确的说一个进程是不能执行的，进程本质上不是一个执行单位，而是一个资源单位（你想开一个工厂，并不是有了这个车间就能生产了，而是应该车间里面至少有一条流水线设备），一个进程内自带一个线程，这个线程才是执行单位，就像是一个车间内只有流水线上的机器才是真正用于生产的。
![avator](https://cdn.nlark.com/yuque/0/2019/png/281865/1554540678105-4d638516-3618-4ab3-a3b0-294db2cba816.png#align=left&display=inline&height=537&originHeight=466&originWidth=720&size=0&status=done&width=829)

其实真正运行进程里面的代码的其实就是这个进程内自带的这个线程，这个也就是这个进程内的主线程。同一进程内的多个线程共享这个进程内的资源，不同进程内的线程资源必然是隔离的。 在一个进程内再次创建一个线程肯定是不需要再次申请内存空间的，那么创建线程的开销自然会比创建进程的开销要小很多，具体小多少，100倍以上。
线程应用场景
比如现在我们需要写一个文本处理工具，像word，notepad++，现在很多文本处理工具都很先进了，他会自动的帮你保存到硬盘，你要考虑的功能至少有4个：

获取用户输入的信息
把用户输入的信息打印到屏幕上
把用户输入的信息保存到硬盘
如果我们让程序串行执行，我都不敢想象这个程序写出来是有多难用，所以，他一定是需要并发执行的，并发的方案无非就是两种，多进程和多线程，我们要处理的信息就是用户输入的内容，如果我们使用多进程，这个通信的过程一定很漫长而且程序很复杂，但是如果我们使用同一个进程内的多线程，那么他们就能共享这个内存空间的数据，我们的处理会变得很容易。其实很多情况下，我们写的程序各部分组件之间都要共享一些数据，所以我们会推荐使用多线程。

# 二 开启线程的两种方式
我们在前面把进程相关的知识打好了基础，那么线程会变得非常简单，开始线程的方式也非常类似，区别就是在于线程的开销比进程小的多。

## 1. 用函数的方式创建线程

In [1]:
from threading import Thread
def task(name):
    print("%s is running" % name)
if __name__ == '__main__':
    t = Thread(target=task, args=('Albert',))
    t.start()  # 同样是发信号，但是线程的开销小的多
    print('主线程')  # 你会看到这一行后打印，这就是与进程的区别

Albert is running主线程



In [2]:
# 线程本来没有父子关系，也就是没有子线程的说法，但是为了区分，我们暂时先把有主线程开启的线程称为“子线程”，接下来请看下面的示例

from threading import Thread
import time
def task(name):
    print("%s is running" % name)
    time.sleep(3)
if __name__ == '__main__':
    t = Thread(target=task, args=('Albert',))
    t.start()  # 同样是发信号，但是线程的开销小的多
    print('主线程')  # 你会看到这一行后打印，这就是与进程的区别

Albert is running主线程



## 2. 用类的方式创建线程

In [3]:
from threading import Thread
import time
import os
def task():
    print("%s is running" % os.getpid())
    time.sleep(3)
if __name__ == '__main__':
    t = Thread(target=task, )
    t.start()
    print('主线程', os.getpid())

16280 is running主线程 16280



In [4]:
# 证明：同一进程内的多个线程共享该进程内的资源
from threading import Thread
import time
x = 999
def task():
    global x
    x = 1
    time.sleep(3)
if __name__ == '__main__':
    t = Thread(target=task, )
    t.start()
    t.join()  # 让主线程等待"子线程"结束，与进程接口相同，我们就不让他睡了
    print('主线程', x)

主线程 1


#  线程互斥锁
以下代码我们希望看到的结果是，100个线程把结果变为0，可结果并不如意

In [5]:
from threading import Thread, Lock
import time
mutex = Lock()
x = 100
def task():
    global x
    mutex.acquire()
    temp = x
    time.sleep(0.1)  # 100个线程需要等待10秒多的时间
    x = temp - 1
    mutex.release()
if __name__ == '__main__':
    start = time.time()
    thread_list = []
    for i in range(100):
        t = Thread(target=task)
        thread_list.append(t)
        t.start()
    for t in thread_list:
        t.join()
    print('主', x)
    print(time.time() - start)

主 0
10.077062129974365


# 六 死锁与递归锁
死锁现象在程序中一旦出现，是很严重的问题，所以我们要尽可能去避免，现在给大家演示一下，人为写bug是怎么写的。

程序执行会进入死锁状态，原因就在于第一个线程和第二个线程，线程创建的快，肯定是第一个创建的线程先起来，他很容易就可以抢到A锁和B锁，然后依次释放并执行f2函数，开始抢B锁，这时其他的线程都在抢A锁，当第一个线程拿到了B锁，肯定也会有一个线程拿到了A锁，这是他们分别需要抢对方手里面的锁，谁都不松手，互相把对方锁死了，那么程序也就卡在原地了。
当有些时候为了保证数据的安全，我们必须要加锁，但是这个处理锁的过程又非常让人头疼，所以，我们应该尽量避免，但是如果躲不过去了，那么我们也能有解决方案。
我们现在用的锁Lock(多进程也是一样)叫做互斥锁，它的一个小缺点就是不能连续acquire，接下来给大家介绍一个递归锁RLock(Recursion Lock)，递归锁的特点就是他可以连续acquire，每次acquire都会给锁的计数+1，这时其他的线程都不能抢，直到锁的计数变为0才可以抢锁，所以问题自然就解决了。

In [ ]:
from threading import Thread, Lock
import time
mutex1 = Lock()
mutex2 = Lock()
class MyThread(Thread):
    def run(self):
        self.f1()
        self.f2()
    def f1(self):
        mutex1.acquire()
        print('%s 拿到了1锁===f1' % self.name)
        mutex2.acquire()
        print('%s 拿到了2锁===f1' % self.name)
        mutex2.release()
        mutex1.release()
    def f2(self):
        mutex2.acquire()
        print('%s 拿到了2锁===f2' % self.name)
        time.sleep(0.1)
        mutex1.acquire()
        print('%s 拿到了1锁===f2' % self.name)
        mutex1.release()
        mutex2.release()
if __name__ == '__main__':
    for i in range(10):
        t = MyThread()
        t.start()
    print('主')

# 七 信号量
我们最开始接触互斥锁的时候使用的大学寝室公用同一个卫生间来说明的，但是除了独用的卫生间，你一定还见过公用的卫生间，它里面会有很多的仓位，假如有10个仓位，如果里面都有人了，那么肯定是不能再进去的，但是只要有一个人出来，那么就能再进去一个人，信号量的作用就是做这样的处理。

In [6]:
# from multiprocessing import Semaphore  # 进程也有信号量
from threading import Thread, Semaphore, current_thread
import time, random
sm = Semaphore(5)  # 卫生间有5个仓位
def go_wc():
    sm.acquire()
    print('%s 上卫生间ing' % current_thread().getName())
    time.sleep(random.randint(1, 3))  # 模拟上卫生间时间
    sm.release()
if __name__ == '__main__':
    for i in range(23):
        t = Thread(target=go_wc)
        t.start()

Thread-110 上卫生间ing
Thread-111 上卫生间ingThread-112 上卫生间ingThread-113 上卫生间ing
Thread-114 上卫生间ing


Thread-115 上卫生间ing
Thread-116 上卫生间ing
Thread-117 上卫生间ing
Thread-118 上卫生间ing
Thread-119 上卫生间ing
Thread-120 上卫生间ing
Thread-121 上卫生间ing
Thread-122 上卫生间ing
Thread-123 上卫生间ing
Thread-124 上卫生间ing
Thread-125 上卫生间ing
Thread-126 上卫生间ing
Thread-127 上卫生间ing
Thread-128 上卫生间ing
Thread-129 上卫生间ing
Thread-130 上卫生间ing
Thread-131 上卫生间ing
Thread-132 上卫生间ing


# 九 进程池与线程池
之前我们写的套接字通信不能实现并发，现在我们可以利用多线程来对他进行修改一下。
服务端代码

In [ ]:
from socket import *
from threading import Thread
def communicate(conn, client_address):
    while True:  
        try:
            data = conn.recv(1024)
            if not data: break
            conn.send(data.upper())
        except ConnectionResetError:
            break
    conn.close()
def server():
    server = socket(AF_INET, SOCK_STREAM)
    server.bind(('127.0.0.1', 8080))
    server.listen(5)
    while True:  # 链接循环
        conn, client_address = server.accept()
        print(client_address)
        t = Thread(target=communicate, args=(conn, client_address))
        t.start()
    server.close()
if __name__ == '__main__':
    server()

In [ ]:
# 客户端代码

from socket import *
client = socket(AF_INET, SOCK_STREAM)
client.connect(('127.0.0.1', 8080))
while True:
    msg = input('>>>: ').strip()
    if not msg: continue
    client.send(msg.encode('utf-8'))
    data = client.recv(1024)
    print(data.decode('utf-8'))
client.close()

我们现在自己测试可能也就会有几个客户端，但是如果是上亿个客户端，虽然开线程的开销小，但是线程也不能无限的开启，随着客户端不断的增多，服务器开启的线程数量会不断的攀升，直到你的服务端机器卡死，瘫痪甚至爆炸(吓唬你们一下啊，不会炸的，但是后果很严重，老板很生气，老板炸了)。
我们不能无限的开进程，也不能无限的开线程，核心的问题就是要控制这个数量，进程或者线程的数量都要在当前环境下可承受的范围之内，这个工作一般是由测试人员和运维人员来负责完成的，这时候我们引入了“池”的概念，使用它就是为了限制并发任务数目，限制我们的机器在一个自己可承受的范围内去并发的执行任务，接下来要说的就是进程池和线程池。
当任务是计算密集型，这是时候应该用多进程，那么也就应用用进程池。当任务是IO密集型，这时候应该用多线程，那么也就应该用线程池。“池”只是一个思路，并不影响多进程或者多线程的使用区别。
进程池与线程池的用法一样，我们先来看一下进程池。

# 十 同步与异步
同步与异步指的是提交任务的两种方式，这与阻塞或者非阻塞是没有直接关系的，可能有些人会有一种山炮逻辑对这两类概念混淆。

>同步调用：提交完任务之后，就在原地等待，直到任务运行完毕后，拿到任务的返回值，
        才继续执行下一行代码。
        
>异步调用：提交完任务之后，不在原地等待，直接执行下一行代码。

## 十二 线程Event
线程event是用来实现在同一个进程内的多个线程协同工作的，比如说有有两个线程，其中一个线程把任务做完了，他必须要发一个通知信号，另外一个线程才能进行工作，这也就是一个A线程工作到了某一个点A线程会通知另外一个B线程开始工作。这种场景就是一个任务要向套接字服务端发连接请求， 但是如果服务端瘫痪，他就会直接报错了，所以我们可以先让发送连接请求的任务等一下，等上面会先有一个【线程1】去检测服务端是否存活， 如果存活的话，【线程1】会发送一个信号告诉【线程2】现在可以去连接服务端了。
如果是多进程，那么只能使用队列了，但是对于多线程，处理起来比较简单，修改全局变量就可以了。
现在有了Event就可以取代这个工作，让他变得更简单。

# 作业和任务：
## 1.  写代码验证在多核CPU前提下对于IO密集型和计算密集型任务多进程和多线程的执行效率，要求如下：         
1 执行效率使用time模块来完成        
2 计算密集型可以用累加或者阶乘，在机器硬件允许的情况下，体现出时间的数量级    
3 IO密集型可以用time来模拟，体现出时间的数量级        
4 计算密集型分别用多进程和多线程实现         
5 IO密集型也分别用多进程和多线程实现
## 2. 写一个简易的爬虫程序，抓取博客园网站首页前10页中所有文章标题带有Python关键字（不区分大小写）的文章，并把文章链接和文章标题存入硬盘，要求如下：         
1 使用正则表达式匹配出需要的数据，然后对数据进行解析      
2 程序中很明显有多个任务，这多个任务必须是异步调用      
3 任务的返回值要及时处理，不能等所有任务都执行完再统一处理     
4 提示信息：可以使用生产者消费者模型来处理       
5 尽可能提升程序的执行效率爬虫储备知识：           requests模块是一个非常强大的爬虫模块，它的基本使用非常简单，但是它是所有爬虫程序的根源，我们只需要使用requests模块里面级简单的功能就能完成，用法可以通过查询得知。
## 3. 使用线程Event写一个监测服务端机器服务状态的程序，要求如下：     
1 模拟服务端机器5秒延迟，客户端每次等待时间为1秒      
2 当用户等待三次还不能开始连接，就返回给用户一个提示信息
